Con este codigo se busca poder "encontrar" el nivel educativo de cada persona, adicionalmente marcar como indefinido a los afiliados que no se lograron identificar en la base de datos.

Para esto se adaptó un script de Yuluka Gigante

In [ ]:
#Importaciones necesarias para que funcione el codigo
import pandas as pd
import pyreadr
import os
import numpy as np
import ast
from datetime import datetime
import json
from collections import Counter
import re

In [ ]:
# Nombre de la carpeta donde antes de ejecutar el script se guardan los datos necesarios
carpeta = "insumos"
if not os.path.exists(carpeta):
    os.makedirs(carpeta)

# Nombre de la carpeta donde despues de ejecutar el script se guardan los datos
carpeta = "salidas"
if not os.path.exists(carpeta):
    os.makedirs(carpeta)

In [ ]:
#Leer el archivo de esa ruta
data = pd.read_excel("insumos/ReporteFinalV2_1.xlsx")

In [ ]:
data.shape

In [ ]:
original_ids = data['Número de documento'].astype(str).tolist()
print(f"🔍 Total IDs originales: {len(original_ids)}")


In [ ]:
def procesar_fila(df, index):
    columnas_filtradas = [col for col in df.columns if col.startswith("Describa los miembros de su familia con los que convive actualmente (Incluyéndose usted):")]

    grupos = sorted(set("_".join(col.split("_")[:2]) for col in columnas_filtradas))  # "columna_opcionX"

    resultados = []

    for grupo in grupos:
        columnas_del_grupo = [col for col in columnas_filtradas if col.startswith(grupo + "_COM-")]
        
        # Verificar si al menos una columna tiene datos válidos
        tiene_dato_valido = any(pd.notna(df.at[index, col]) for col in columnas_del_grupo)
        
        if tiene_dato_valido:
            # Construir el JSON con todas las claves, convirtiendo los valores a tipos estándar
            json_obj = {col.split("_")[-1]: df.at[index, col] if pd.notna(df.at[index, col]) else None for col in columnas_del_grupo}

            # Convertir valores NumPy a tipos nativos de Python
            json_obj = {k: (int(v) if isinstance(v, np.integer) else float(v) if isinstance(v, np.floating) else v) for k, v in json_obj.items()}
            
            resultados.append(json_obj)

    return resultados


In [ ]:
resultado = procesar_fila(data, 124)
print(resultado)

In [ ]:
found_members = {}  # Diccionario para almacenar las personas encontradas

for index in range(len(data)):  # Iteramos sobre cada fila del DataFrame `data`
    # Construimos una cadena de información con el número de documento, nombres y apellidos
    person_info = f"{data['Número de documento'][index]} - {data['Nombres:'][index]} {data['Apellidos:'][index]}"

    # Obtenemos el sexo y la fecha de nacimiento de la persona
    sex = data['¿Cómo se reconoce?'][index]
    birth_datestr = data['Fecha de nacimiento:'][index]
    birth_date = datetime.strptime(birth_datestr, "%d/%m/%Y")  # Convertimos la fecha a objeto datetime

    # Obtenemos la fecha en la que se realizó la encuesta
    survey_time_start_date = datetime.strptime(data['TimeStart'][index], "%Y-%m-%d %H:%M:%S")

    # Calculamos la edad en el momento de la encuesta
    person_age = str(survey_time_start_date.year - birth_date.year - 
                     ((survey_time_start_date.month, survey_time_start_date.day) < 
                      (birth_date.month, birth_date.day)))

    # Convertimos el sexo reportado en el formulario a una forma esperada
    if sex == 'Hombre':
        expected_sex = 'Masculino'
    elif sex == 'Mujer':
        expected_sex = 'Femenino'

    # Procesamos la fila para obtener la información de los miembros de la familia
    person_family = procesar_fila(data, index)

    # Iteramos sobre los miembros de la familia de la persona encuestada
    for member in person_family:
        # Obtenemos la información del miembro de la familia
        member_sex = member['COM-23'].strip().lower()  # Normalizamos el formato del sexo
        member_age = int(member['COM-24']) if member['COM-24'] is not None else -1  # Convertimos la edad a entero (-1 si es nulo)
        member_work = member['COM-27'].strip().lower()  # Normalizamos el formato del tipo de trabajo

        # Convertimos las variables a minúsculas y enteros para evitar inconsistencias
        expected_sex = expected_sex.lower()
        person_age = int(person_age)

        # Verificamos si el miembro de la familia cumple con los criterios esperados
        if member_sex == expected_sex and member_age == person_age and member_work == 'trabajo formal':
            # Si encontramos coincidencia, almacenamos la información en `found_members`
            found_members[person_info] = member
            print(f"✅ ¡Encontrado!: {person_info} -> {member}")

            # Asignamos el nivel educativo del miembro a la persona encuestada en el DataFrame
            data.at[index, 'Nivel Educativo'] = member.get('COM-25', 'nivel educativo')

            # Marcamos el estado de la persona encuestada como "Afiliado"
            data.at[index, 'Estado'] = "Afiliado"
            break  # Salimos del bucle porque ya encontramos una coincidencia

# Mostramos cuántas personas han sido encontradas
print(f"Found: {len(found_members)} de {len(data)}")


In [ ]:
found_keys = {key.split(' - ')[0] for key in found_members.keys()}
print(f"✅ Total IDs encontrados: {len(found_keys)}")
not_found = data[~data['Número de documento'].astype(str).isin(found_keys)]
not_found_ids = data[~data['Número de documento'].astype(str).isin(found_keys)]['Número de documento'].astype(str).tolist()
print(f"❌ Total IDs no encontrados: {len(not_found_ids)}")

print("-" * 100)
found_counter = Counter(found_keys)
duplicated_found = {key: val for key, val in found_counter.items() if val > 1}

not_found_counter = Counter(not_found_ids)
duplicated_not_found = {key: val for key, val in not_found_counter.items() if val > 1}

print(f"🔎 IDs duplicados en Found: {len(duplicated_found)}")
print(f"🛑 IDs duplicados en Not Found: {len(duplicated_not_found)}")

# Mostrar algunos ejemplos de duplicados
print(f"Ejemplo duplicados en Found: {list(duplicated_found.items())[:5]}")
print(f"Ejemplo duplicados en Not Found: {list(duplicated_not_found.items())[:5]}")

print("-" * 100)

# Encontrar IDs que están en ambos grupos
intersection_ids = set(found_keys) & set(not_found_ids)

print(f"⚠️ IDs que aparecen en ambos grupos: {len(intersection_ids)}")
print(f"Ejemplo de IDs en ambos grupos: {list(intersection_ids)[:5]}")

print("-" * 100)

# Contamos cuántas veces aparece cada ID en `data`
data_counter = Counter(original_ids)
duplicated_data = {key: val for key, val in data_counter.items() if val > 1}

print(f"🔍 IDs duplicados en `data`: {len(duplicated_data)}")
print(f"Ejemplo de IDs duplicados en `data`: {list(duplicated_data.items())[:5]}")


In [ ]:
# Asignar "Indefinido" a los registros no encontrados
data.loc[data['Número de documento'].astype(str).isin(not_found_ids), 'Estado'] = 'Indefinido'

In [ ]:
data.to_excel('salidas/ReporteFinalV2_1_ACTUALIZADO.xlsx', index=False)

In [ ]:
# Filtramos solo los registros que tienen IDs duplicados
duplicated_rows = data[data['Número de documento'].astype(str).isin(duplicated_data)]

# Ordenamos por ID para facilitar la comparación
duplicated_rows = duplicated_rows.sort_values(by='Número de documento')

# Guardamos el resultado en un Excel para revisar manualmente (opcional)
duplicated_rows.to_excel('salidas/duplicated_records.xlsx', index=False)

# Mostramos las primeras filas para inspección rápida
print(duplicated_rows)


In [ ]:
not_found.to_excel('salidas/not_found_members.xlsx', index=False)

In [ ]:
not_found.shape

In [ ]:
data.shape

In [ ]:
for index in range(len(data)):
    person_family = procesar_fila(data, index)

    print(f"Total miembros de familia: {len(person_family)}")

    for member in person_family:
        print(f"{member['COM-20']} - {member['COM-21']}")